# neural network belief propagation decode quantum code: database generating  
This note illustrate database and relate information generating of `[1]`. Generate the 2D Toric code syndrome data and posterior probabilistic distribution(PPD) for `[1]`.

In [1]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt
import torch 
import os
import sys

np.set_printoptions(threshold = 1e6)

### **First Part: generate the parity check matrix for 2D Toric code**

In [ ]:
def generate_PCM(k, L):
    j = 0
    H = np.zeros([(2 * L * L - 2), (4 * L * L)])
    for i in range(int(k / 2)):
        H[i][j] = H[i][j + L] = H[i][(j + 2 * L) % (2 * L * L)] = 1
        H[i + int(k / 2)][2 * L * L + j] = H[i + int(k / 2)][2 * L * L + j + L] \
        = H[i + int(k / 2)][2 * L * L + (j - L) % (2 * L * L)]  = 1
        if (j + L + 1) % L == 0:
            H[i][j + 1] = 1
        else:
            H[i][j + L + 1] = 1
        if j % L != 0:
            H[i + int(k / 2)][2 * L * L + j - 1] = 1
        else:
            H[i + int(k / 2)][2 * L * L + (j - 1 + L)] = 1
        if (j + 1) % L == 0:
            j = j + L
        j += 1
    return H

### **Second Part: generate the H_prep matrix**

In [ ]:
class H_Prep():
    def __init__(self, H):
        self.H = H
        self.rows = int(H.shape[0])
        self.cols = int(H.shape[1])
        self.H_prep = H
        
    def Get_Identity(self):
        H_prime = self.H
        exchange = []
        for i in range(self.rows):
            if H_prime[i, i] != 1:
                for j in range(i, self.cols):
                    if H_prime[i, j] == 1:
                        H_prime[:, [i, j]] = H_prime[:, [j, i]]
                        exchange.append((i,j))
        exchange.reverse()
        return H_prime, exchange

### **Thrid Part: **

In [ ]:
    def get_H_Prep(self):
        self.H_prep, exchange = self.Get_Identity()
        self.H_prep = np.concatenate([self.H_prep[:, self.rows : self.cols], np.eye(self.cols - self.rows)], axis = 0)
        for item in exchange:
            self.H_prep[[item[0], item[1]], :] = self.H_prep[[item[1], item[0]], :]
        self.H_prep = np.concatenate([self.H_prep[int(self.cols / 2) : self.cols, :], \
                                                  self.H_prep[0 : int(self.cols / 2), :]], axis = 0)
        self.H_prep = self.H_prep.T
        return self.H_prep
    
    def symplectic_product(a, b):
        rows, cols = a.shape
        return np.dot(a, np.concatenate([b[:, int(cols / 2) : cols], b[:, 0 : int(cols / 2)]], axis = 1).T) % 2
    
    def get_logical(self):
        self.H_prep = self.get_H_Prep()
        rows, cols = self.H_prep.shape
        logical = []
        for i in range(rows):
            if self.H_prep[i, :] not in logical:
                for j in range(i + 1, rows):
                    if self.H_prep[j, :] not in logical:
                        if self.symplectic_product(self.H_prep[i, :], self.H_prep[j, :]) == 1:
                            logical.append(self.H_prep[i, :])
                            logical.append(self.H_prep[j, :])
                            for k in range(j + 1, rows):
                                if self.H_prep[k, :] not in logical:
                                    if self.symplectic_product(self.H_prep[i, :], self.H_prep[k, :]) == 1:
                                        self.H_prep[k, :] += self.H_prep[j, :]
                            for m in range(i + 1, rows):
                                if self.H_prep[m, :] not in logical:
                                    if self.symplectic_product(self.H_prep[j, :], self.H_prep[m, :]) == 1:
                                        self.H_prep[m, :] += self.H_prep[i, :]
                            break
        logical = np.array(logical)
        return logical

### **Forth Part: generating the syndrome and corresponding PPD data**

In [ ]:
def gen_syn(P, L, H, run):
    dataset = []
    err = np.zeros((1, 4 * L * L))
    rows, cols = H.shape
    for i in range(run):
        p = random.sample(P, 1)[0]
        prior = np.full((1, 4 * L * L), math.log((1 - p) / p))
        for j in range(2 * L * L):
            a, b = np.random.random(), np.random.random()
            if a < p:
                err[0, j] = 1 #X error
            if b < p:
                err[0, j + 2 * L * L] = 1 #Z error
        syn = (np.dot(H, err.T) % 2).T
        syn = np.concatenate([syn, np.zeros((1, cols - rows))], axis = 1)
        dataset.append(torch.from_numpy(np.concatenate([prior, syn], axis = 0)))
        dataset.append(torch.from_numpy(err))
        err = np.zeros((1, 4 * L * L))
    return dataset

### **Ffith Part: testing the program**

In [ ]:
if __name__ == '__main__':
    L = 12
    H = generate_PCM(2 * L * L - 2, L)
    P = [0.01]
    dataset = gen_syn(P, L, H, 120000)
    print(sys.getsizeof(dataset))

### **Reference**

`[1]`: Ye-Hua Liu and David Poulin, “Neural belief-propagation decoders for quantum error-correcting codes,” arXiv preprint arXiv:1811.07835 (2018).  